# Страховой скоринг.Финальное решение
## Классификация клиентов по уровню благонадежности

### Инструкция
Загрузите данные, приведите их к числовым, заполните пропуски, нормализуйте данные и оптимизируйте память.

Сформируйте параллельный **ансамбль из CatBoost, градиентного бустинга, XGBoost и LightGBM**. Используйте лучшие гиперпараметры, подобранные ранее, или найдите их через перекрестную проверку. Итоговое решение рассчитайте на основании самого точного предсказания класса у определенной модели ансамбля: выберите для каждого класса модель, которая предсказывает его лучше всего.

Проведите расчеты и выгрузите результат в виде submission.csv

Данные:
* http://video.ittensive.com/machine-learning/prudential/train.csv.gz
* http://video.ittensive.com/machine-learning/prudential/test.csv.gz
* http://video.ittensive.com/machine-learning/prudential/sample_submission.csv.gz

Итоговый файл с кодом (.py или .ipynb) выложите в github с портфолио.

### *решение переделано на основе материалов курса*

Выполнялось на Python 3.11.  CatBoost пока не сделали сборку под 3.11, поэтому исключаем CatBoost из ансамбля


### Подключение библиотек

In [61]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix
# from catboost import Pool, CatBoostClassifier     # исключен из ансамбля
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn import preprocessing

### Загрузка данных

In [62]:
data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


### Предобработка данных

* Выделение из параметра Product_Info_2 символьной и числовой составляющих
* Преобразование в one-hot-vector (единичные векторы)
* заполнение пропусков -1 для увеличения расстояния


In [63]:
def data_preprocess (df):
    df["Product_Info_2_1"] = df["Product_Info_2"].str.slice(0, 1)
    df["Product_Info_2_2"] = pd.to_numeric(df["Product_Info_2"].str.slice(1, 2))
    df.drop(labels=["Product_Info_2"], axis=1, inplace=True)
    
    df = pd.get_dummies(data=df,prefix='Product_Info_2_1')
    df.fillna(value=-1, inplace=True)
    
    #     data["Response"] = data["Response"] - 1
    return df

In [64]:
data = data_preprocess(data)
data["Response"] = data["Response"] - 1   # перевод номеров в 0-based индексы
print(data.head())

   Id  Product_Info_1  Product_Info_3  Product_Info_4  Product_Info_5  \
0   2               1              10        0.076923               2   
1   5               1              26        0.076923               2   
2   6               1              26        0.076923               2   
3   7               1              10        0.487179               2   
4   8               1              26        0.230769               2   

   Product_Info_6  Product_Info_7   Ins_Age        Ht        Wt  ...  \
0               1               1  0.641791  0.581818  0.148536  ...   
1               3               1  0.059701  0.600000  0.131799  ...   
2               3               1  0.029851  0.745455  0.288703  ...   
3               3               1  0.164179  0.672727  0.205021  ...   
4               3               1  0.417910  0.654545  0.234310  ...   

   Medical_Keyword_46  Medical_Keyword_47  Medical_Keyword_48  Response  \
0                   0                   0            

In [65]:
data.Response.unique()

array([7, 3, 0, 5, 1, 6, 2, 4], dtype=int64)

### Набор столбцов для расчета

In [66]:
columns_groups = ["Insurance_History", "InsurеdInfo", "Medical_Keyword",
                  "Family_Hist", "Medical_History", "Product_Info"]
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print (columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

### Нормализация данных

In [67]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data,
                                                     columns=columns)))
columns_transformed = data_transformed.columns
data_transformed["Response"] = data["Response"]

### Оптимизация памяти

In [68]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [69]:
data_transformed = reduce_mem_usage(data_transformed)
print (data_transformed.info())

Потребление памяти меньше на 40.49 Мб (минус 75.1 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 119 entries, 0 to Response
dtypes: float16(118), int8(1)
memory usage: 13.4 MB
None


### Построение базовых моделей

In [70]:
x = pd.DataFrame(data_transformed, columns=columns_transformed)

Построим XGBoost модель

In [71]:
model_xgb = XGBClassifier(max_depth=17, max_features=27,  n_estimators=76, learning_rate=.1, 
                          booster='gbtree', min_samples_leaf=20)
model_xgb.fit(x, data['Response'])

[18:11:53] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "max_features", "min_samples_leaf" } are not used.



XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=17, max_features=27,
              max_leaves=None, min_child_weight=None, min_samples_leaf=20,
              missing=nan, monotone_constraints=None, n_estimators=76,
              n_jobs=None, num_parallel_tree=None, ...)

Модель Градиентный бустинг

In [72]:
model_gbc = GradientBoostingClassifier(random_state=17, max_depth=13, max_features=26, min_samples_leaf=21, n_estimators=75)
model_gbc.fit(x, data['Response'])

GradientBoostingClassifier(max_depth=13, max_features=26, min_samples_leaf=21,
                           n_estimators=75, random_state=17)

Модель LightGBM

In [73]:
model_lgbm = lgb.LGBMRegressor(random_state=17, max_depth=18, min_child_samples=17, num_leaves=35, n_estimators=10000)
model_lgbm.fit(x, data['Response'])

LGBMRegressor(max_depth=18, min_child_samples=17, n_estimators=10000,
              num_leaves=35, random_state=17)

### Загрузка данных для расчета

In [74]:
data_test = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/test.csv.gz")
data_test = data_preprocess(data_test)
print(data_test.head(2))


   Id  Product_Info_1  Product_Info_3  Product_Info_4  Product_Info_5  \
0   1               1              26        0.487179               2   
1   3               1              26        0.076923               2   

   Product_Info_6  Product_Info_7   Ins_Age        Ht        Wt  ...  \
0               3               1  0.611940  0.781818  0.338912  ...   
1               3               1  0.626866  0.727273  0.311715  ...   

   Medical_Keyword_45  Medical_Keyword_46  Medical_Keyword_47  \
0                   0                   0                   0   
1                   0                   0                   0   

   Medical_Keyword_48  Product_Info_2_2  Product_Info_2_1_A  \
0                   0                 3                   0   
1                   0                 2                   1   

   Product_Info_2_1_B  Product_Info_2_1_C  Product_Info_2_1_D  \
0                   0                   0                   1   
1                   0                   0      

In [75]:
data_test = reduce_mem_usage(data_test)
print(columns)

Потребление памяти меньше на 16.34 Мб (минус 84.9 %)
['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keywor

In [76]:
data_test_transformed = pd.DataFrame(scaler.transform(pd.DataFrame(data_test,
                                                     columns=columns)))
print (data_test_transformed.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19765 entries, 0 to 19764
Columns: 118 entries, 0 to 117
dtypes: float64(118)
memory usage: 17.8 MB
None


### Расчет предказаний

In [77]:
data_test["target_xgb"] = model_xgb.predict(data_test_transformed)

C:\Users\Sovtsov\AppData\Local\Temp\ipykernel_11044\1008409102.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_test["target_xgb"] = model_xgb.predict(data_test_transformed)


In [79]:
data_test["target_gbc"] = model_gbc.predict(data_test_transformed)

C:\Users\Sovtsov\AppData\Local\Temp\ipykernel_11044\1559723127.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_test["target_gbc"] = model_gbc.predict(data_test_transformed)


In [80]:
data_test["target_lgbm"] = np.round(model_lgbm.predict(data_test_transformed)).astype("int8")

C:\Users\Sovtsov\AppData\Local\Temp\ipykernel_11044\864458132.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_test["target_lgbm"] = np.round(model_lgbm.predict(data_test_transformed)).astype("int8")


In [81]:
print(data_test.head())

   Id  Product_Info_1  Product_Info_3  Product_Info_4  Product_Info_5  \
0   1               1              26        0.487061               2   
1   3               1              26        0.076904               2   
2   4               1              26        0.144653               2   
3   9               1              26        0.151733               2   
4  12               1              26        0.076904               2   

   Product_Info_6  Product_Info_7   Ins_Age        Ht        Wt  ...  \
0               3               1  0.611816  0.781738  0.338867  ...   
1               3               1  0.626953  0.727051  0.311768  ...   
2               3               1  0.582031  0.708984  0.320068  ...   
3               1               1  0.522461  0.654785  0.267822  ...   
4               3               1  0.298584  0.672852  0.246826  ...   

   Medical_Keyword_48  Product_Info_2_2  Product_Info_2_1_A  \
0                   0                 3                   0   
1 

## Получение сводного значения класса (голосование в ансамбле)
Классы смещены на 1: начинаются от 0 и заканчиваются 7. 

Перекрестная проверка всех данных не выполнялась, из-за недостатка вычислительных ресурсов.

Матрицы ошибок моделей с использованными гиперпараметрами были рассчитаны в предыдущих практических примерах.

Ориентируясь на них, можно составить следующее правило агрегации: \
класс 5 - lgbm \
класс 2,3 - xgb \
остальные - gbc


In [82]:
def vote_class (x):
    if x.target_xgb in [2,3]:
        class_ = x.target_xgb
    elif x.target_lgbm == 5:
        class_ = x.target_lgbm
    else:
        class_ = x.target_gbc
    x["Response"] = class_ + 1    # возвращаем значения к первоначальным номерам
    return x

In [83]:
data_test = data_test.apply(vote_class, axis=1)
print (data_test.head())

     Id  Product_Info_1  Product_Info_3  Product_Info_4  Product_Info_5  \
0   1.0             1.0            26.0        0.487061             2.0   
1   3.0             1.0            26.0        0.076904             2.0   
2   4.0             1.0            26.0        0.144653             2.0   
3   9.0             1.0            26.0        0.151733             2.0   
4  12.0             1.0            26.0        0.076904             2.0   

   Product_Info_6  Product_Info_7   Ins_Age        Ht        Wt  ...  \
0             3.0             1.0  0.611816  0.781738  0.338867  ...   
1             3.0             1.0  0.626953  0.727051  0.311768  ...   
2             3.0             1.0  0.582031  0.708984  0.320068  ...   
3             1.0             1.0  0.522461  0.654785  0.267822  ...   
4             3.0             1.0  0.298584  0.672852  0.246826  ...   

   Product_Info_2_2  Product_Info_2_1_A  Product_Info_2_1_B  \
0               3.0                 0.0              

### Самопроверка точности модели
Рассчитаем точность классификации на обучающих данных

In [84]:
data_copy = data_transformed.copy()
x_copy = pd.DataFrame(data_copy, columns=columns_transformed)
data_copy["target_xgb"] = model_xgb.predict(x_copy)
data_copy["target_gbc"] = model_gbc.predict(x_copy)
data_copy["target_lgbm"] = np.round(model_lgbm.predict(x_copy)).astype("int8")  # прогноз возвращает не целое

In [85]:
# список для записи лучшей модели в классе
class_target = ["target_gbc"]*8


In [86]:
# выбираем "угадавшую" модель
def vote_class_enumerate (x):
    for _,target in enumerate(class_target):
        if x[target] == _:
            x["Response"] = x[target]
            break
    return x

In [87]:
# вычисляем наилучшую оценку для модели 
kappa_min = 0
for target_model in ["xgb", "gbc", "lgbm"]:
    print ("Проверяем модель:", target_model)
    target_model = "target_" + target_model
    for c in range(0,8):
        target_model_prev = class_target[c]
        class_target[c] = target_model
        data_copy = data_copy.apply(vote_class_enumerate, axis=1)
        kappa = cohen_kappa_score(data_copy["Response"], 
                data["Response"], weights='quadratic')
        if kappa > kappa_min:
            kappa_min = kappa
        else:
            class_target[c] = target_model_prev
    print ("Максимальная оценка:", kappa_min)
print (class_target)

Проверяем модель: xgb
Максимальная оценка: 0.9276509716965988
Проверяем модель: gbc
Максимальная оценка: 0.9276695961815296
Проверяем модель: lgbm
Максимальная оценка: 0.9859283142567946
['target_xgb', 'target_lgbm', 'target_lgbm', 'target_lgbm', 'target_lgbm', 'target_lgbm', 'target_lgbm', 'target_lgbm']


In [88]:
data_copy = data_copy.apply(vote_class_enumerate, axis=1)

In [89]:
print ("Результат:",
       round(cohen_kappa_score(data_copy["Response"],
                    data["Response"], weights='quadratic'), 3))
print (confusion_matrix(data_copy["Response"], data["Response"]))

Результат: 0.986
[[ 5546     0     0     0     0     0     0     0]
 [  385  5672     1     0     0     0     0     0]
 [  117   743   972    17     0     0     0     0]
 [   19   115    34  1332    43     2     0     0]
 [    7    21     5    68  5266   147     0     1]
 [   31     1     1     9   122 10915   207     5]
 [   23     0     0     2     1   167  7801   868]
 [   79     0     0     0     0     2    19 18615]]


### Формирование и выгрузка результатов
Загрузим файл с примером, заменим в нем результаты и сохраним.

Число строк в файле будет равно размену набора данных + 1 заголовочная строка.

In [90]:
submission = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/sample_submission.csv.gz")
submission["Response"] = data_test["Response"].astype("int8")
submission.to_csv("submission.csv", index=False)
print (len(submission["Response"]) + 1)

19766
